In [1]:
import json
import pandas as pd

import PySAM
import PySAM.Singleowner as single_owner
import PySAM.Pvsamv1 as pv_model
import PySAM.Grid as grid
import PySAM.Utilityrate5 as utility_rate
import PySAM.ResourceTools as tools

import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import seaborn as sns

In [27]:
df = pd.read_excel("Agrivoltaics Row Spacing Cost Curve.xlsx")
df = df.set_index("Index")

In [28]:
print(df.index)
cost_per_watt = df.loc["31.7ft", 0.9]
print(cost_per_watt)

Index(['16.7ft', '21.7ft', '26.7ft', '31.7ft', '36.7ft', '41.7ft', '46.7ft',
       '51.7ft', '56.7ft', '61.7ft', '66.7ft', '71.7ft', '76.7ft',      nan,
            nan,      nan],
      dtype='object', name='Index')
1.4921497785262972


In [30]:
import json
import pandas as pd
import os

import PySAM
import PySAM.Singleowner as single_owner
import PySAM.Pvsamv1 as pv_model
import PySAM.Grid as grid
import PySAM.Utilityrate5 as utility_rate
import PySAM.ResourceTools as tools

from functools import partial
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import multiprocessing as mp
import numpy as np
import seaborn as sns

def get_cmod_json(json_file_path):
    with open(json_file_path, 'r') as f:
        dict = json.load(f)
    return dict

#ppa_prices = [x / 1000.0 for x in range(0, 70, 2)] # Range only accepts integers. PPA prices of $0.0/kWh to $0.2/kWh in increments of $0.005
#crop_profit = range(0, 500000, 10000) # $5,000/acre * 100 acres in $100/acre increments


locs = pd.read_excel("Colorado Counties Crop Count for Economic Analysis.xlsx", header=1)

equipment_data = pd.read_excel("equipment_width_and_crop_profit_factor.xlsx", header=1)
equipment_data = equipment_data.set_index("Row spacing (ft)")
crop_multipliers = equipment_data[['Onions - profit', 'Potatoes - profit', 'Sugar Beets - profit', 'Wheat - profit', 'Equipment agnostic - profit', 'Open air - profit']].copy()
crop_multipliers = crop_multipliers.dropna()
crops = ["Onions"]

config = '31.7ft'
co_idx = 0

acres = 160
#min_profit = -1000 * acres
#max_profit = 1000 * acres

min_profit = 0 * acres
max_profit = 1000 * acres

ppa_prices =  [0.0515]
crop_profit = range(min_profit, max_profit + 1, 50 * acres)

prices =  [x / 10.0 for x in range(5, 26, 2)] #0.5 to #2.5

cost_matrix = pd.read_excel("Agrivoltaics Row Spacing Cost Curve.xlsx")
cost_matrix = cost_matrix.set_index("Index")

full_data = []
columns = ["County", "Lat", "Long", "Config", "Crop", "Solar capacity", "CAPEX", "PPA price", "Open Air Crop Profit", "Specific Crop Profit", \
            "NPV", "IRR", "Capacity Factor", "Annual Energy Production (year 1)", "Energy Value (year 1)", "Discounted Payback", "$/W", "GCR Cost Multiplier"]

data_folder = "input_json/"
output_folder = "pv_json/"

pv = pv_model.default("FlatPlatePVSingleOwner")

pv_json = get_cmod_json(data_folder + config + "_pvsamv1.json")
so_json = get_cmod_json(data_folder + config + "_singleowner.json")

co_name = locs.loc[co_idx]["COUNTY"]
co_lat = locs.loc[co_idx ]["CENT_LAT"]
co_long = locs.loc[co_idx ]["CENT_LONG"]

filename = output_folder + co_name + "_" + str(config) + ".json"

print(config, co_name)
gen = []
annual_energy = 0
cf = 0
if not os.path.isfile(filename):

    for k, v in pv_json.items():
        if "adjust_" in k:
            k = k.replace("adjust_", "")
        try:
            pv.value(k, v)
        except AttributeError:
            print("Error ", k)
    
    pv.SolarResource.solar_resource_file = "weather_data/nsrdb_" + str(co_lat) + "_" + str(co_long) +"_psm3-tmy_60_tmy.csv"

    pv.execute()
    gen = pv.Outputs.gen
    annual_energy = pv.Outputs.annual_energy
    cf = pv.Outputs.capacity_factor

    with open(filename, "w") as f:
        data = {"gen" : pv.Outputs.gen,
                "cf" : pv.Outputs.capacity_factor,
                "annual_energy" : pv.Outputs.annual_energy}
        json.dump(data, f, indent=4)

else:
    with open(filename, "r") as f:
        data = json.load(f)
        gen = data["gen"]
        cf = data["cf"]
        annual_energy = data["annual_energy"]

for cost in prices:
    for price in ppa_prices:
        for profits in crop_profit:
            for crop in crops:
                so = single_owner.default("FlatPlatePVSingleOwner")

                for k, v in pv_json.items():
                    if "adjust_" in k:
                        k = k.replace("adjust_", "")
                    try:
                        pv.value(k, v)
                    except:
                        pass 

                for k, v in so_json.items():
                    try:
                        so.value(k, v)
                    except AttributeError:
                        print("Error ", k)

                so.SystemOutput.gen = gen
                so.SystemOutput.system_pre_curtailment_kwac = gen
                so.FinancialParameters.system_capacity = pv.SystemDesign.system_capacity

                print(config, cost)
                cost_per_watt = cost_matrix.loc[config, cost]
                so.SystemCosts.total_installed_cost = cost_per_watt * pv.SystemDesign.system_capacity

                so.Revenue.ppa_price_input = [price]
                
                row_float = float(config[0:4])
                multiplier = crop_multipliers.loc[row_float][crop + " - profit"]
                specific_profit = profits * multiplier
                so.SystemCosts.om_fixed = [-1.0 * specific_profit]

                so.execute()

                # ["County", "Lat", "Long", "Config", "Crop", "Solar capacity", "CAPEX", "PPA price", "Open Air Crop Profit", "Specific Crop Profit" \
                # "NPV", "IRR", "Capacity Factor", "Annual Energy Production (year 1)", "Energy Value (year 1)", "Discounted Payback"]

                npvs = so.Outputs.cf_project_return_aftertax_npv
                payback = 99
                for i, npv in enumerate(npvs):
                    if npv > 0:
                        payback = i
                        break

                run_data = [co_name, # County
                            co_lat, # Lat
                            co_long, # Long
                            config, # Config
                            crop, # Crop
                            so.SystemOutput.system_capacity, # Solar capacity
                            so.SystemCosts.total_installed_cost + so.FinancialParameters.construction_financing_cost, # CAPEX
                            price, # PPA price
                            profits, # Open Air Crop Profit
                            specific_profit, # Specific Crop Profit
                            so.Outputs.project_return_aftertax_npv, #NPV
                            so.Outputs.project_return_aftertax_irr, #IRR
                            cf, # Capacity Factor - DC
                            annual_energy, # Annual Energy
                            list(so.Outputs.cf_energy_value)[1], # Energy Value
                            payback, # Discounted Payback
                            cost_per_watt,
                            cost]

                print(run_data)
                full_data.append(run_data)

df = pd.DataFrame(data=full_data, columns=columns)

print(df)

31.7ft ADAMS
31.7ft 0.5
['ADAMS', 39.8736335787425, -104.337935219918, '31.7ft', 'Onions', 23866.07328, 914450.7001625492, 0.0515, 0, 0.0, 23957615.57436915, 609.2626020428918, 23.679162045427596, 49505266.81286086, 2550206.2661094023, 1, 1.4069497785262972, 0.5]
31.7ft 0.5
['ADAMS', 39.8736335787425, -104.337935219918, '31.7ft', 'Onions', 23866.07328, 914450.7001625492, 0.0515, 8000, 5030.30303030303, 24003247.44725845, 610.0838616099588, 23.679162045427596, 49505266.81286086, 2550206.2661094023, 1, 1.4069497785262972, 0.5]
31.7ft 0.5
['ADAMS', 39.8736335787425, -104.337935219918, '31.7ft', 'Onions', 23866.07328, 914450.7001625492, 0.0515, 16000, 10060.60606060606, 24048879.32014775, 610.9051159068939, 23.679162045427596, 49505266.81286086, 2550206.2661094023, 1, 1.4069497785262972, 0.5]
31.7ft 0.5
['ADAMS', 39.8736335787425, -104.337935219918, '31.7ft', 'Onions', 23866.07328, 914450.7001625492, 0.0515, 24000, 15090.90909090909, 24094511.19303704, 611.7263649553627, 23.679162045427596